# BiLSTM-CRF
 bilstm-crf 模型主要用于解决命名实体识别问题，
 
 crf 和 bilstm 是该模型的两个层。bilstm 将输入的序列文本转化为每个词对应的序列标记的类别的概率。crf则根据这些类别的概率根据损失函数将真实路径的概率最大化。
 
 

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

In [18]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from crf import CRF

In [19]:
class MyBiLSTMCRF:
    def __init__(self, vocabSize, maxLen, tagIndexDict,tagSum,sequenceLengths=None,vecSize=100):
        self.vocabSize = vocabSize
        self.vecSize = vecSize
        self.maxLen = maxLen
        self.tagSum = tagSum
        self.sequenceLengths=sequenceLengths
        self.tagIndexDict=tagIndexDict

        self.buildBiLSTMCRF()

    def getTransParam(self,y,tagIndexDict):
        self.trainY=y
        yList=self.trainY.tolist()
        transParam=np.zeros([len(list(tagIndexDict.keys())),len(list(tagIndexDict.keys()))])
        for rowI in range(len(yList)):
    
            for colI in range(len(yList[rowI])-1):
                transParam[yList[rowI][colI]][yList[rowI][colI+1]]+=1
        for rowI in range(transParam.shape[0]):
            transParam[rowI]=transParam[rowI]/np.sum(transParam[rowI])
        return transParam
    
    def buildBiLSTMCRF(self):

        myModel=Sequential()
        myModel.add(tf.keras.layers.Input(shape=(self.maxLen,)))
        myModel.add(tf.keras.layers.Embedding(self.vocabSize, self.vecSize))
        myModel.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                    self.tagSum, return_sequences=True, activation="tanh"), merge_mode='sum'))
        myModel.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                    self.tagSum, return_sequences=True, activation="softmax"), merge_mode='sum'))
        crf=CRF(self.tagSum,name='crf_layer')
        myModel.add(crf)
        myModel.compile('adam',loss={'crf_layer': crf.get_loss})
        self.myBiLSTMCRF=myModel
        
    def fit(self,X,y,epochs=100,transParam=None):
        if len(y.shape)==3:
            y=np.argmax(y,axis=-1)
        if self.sequenceLengths is None:
            self.sequenceLengths=[row.shape[0] for row in y]
            
        self.transParam = self.getTransParam(y, self.tagIndexDict)
            
        log_dir = "logs"
        tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

        history=self.myBiLSTMCRF.fit(X,y,epochs=epochs,callbacks=[tensorboard_callback])

        return history

    def predict(self,X):
        preYArr=self.myBiLSTMCRF.predict(X)
        return preYArr

In [8]:
import pandas as pd

In [10]:
myDf=pd.read_csv("../data/test_bilstm_crf.csv")

In [11]:
myDf["text"]=myDf["text"].apply(lambda x:x+" <end>")
myDf["tag"]=myDf["tag"].apply(lambda x:x+" END")
myDf[:1]


,text,tag
0,将 军 百 战 死 <end>,B I B I S END


In [12]:
wordIndexDict={"<pad>":0}
wi=1
for row in myDf["text"].values.tolist():
    if type(row)==float:
        print(row)
        break
    for word in row.split(" "):
        if word not in wordIndexDict:
            wordIndexDict[word]=wi
            wi+=1
vocabSize=wi

In [13]:
maxLen=max(len(row) for row in myDf["text"].values.tolist())
sequenceLengths=[len(row) for row in myDf["text"].values.tolist()]

In [14]:
myDf["text"]=myDf["text"].apply(lambda x:[wordIndexDict[word] for word in x.split()])

In [15]:
X=tf.keras.preprocessing.sequence.pad_sequences(myDf["text"],
                                                value=wordIndexDict["<pad>"],
                                                padding='post',
                                                maxlen=maxLen)
X

array([[ 1,  2,  3,  4,  5,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  7,  8,  9,  4, 10,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  1,  2,  8,  9,  4, 11,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 1,  2,  4,  1,  2,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0]])

In [16]:
import tqdm
import re

myDf["tag"]=myDf["tag"].apply(lambda x:re.sub("\-\S+","",x))

tagIndexDict = {"PAD": 0}
ti = 1
for row in tqdm.tqdm(myDf["tag"].values.tolist()):
    for tag in row.split(" "):
        if tag not in tagIndexDict:
            tagIndexDict[tag] = ti
            ti += 1
tagSum = len(list(tagIndexDict.keys()))
myDf["tag"] = myDf["tag"].apply(lambda x:x.split()+["PAD" for i in range(maxLen-len(x.split()))])
myDf["tag"] = myDf["tag"].apply(lambda x:[tagIndexDict[tagItem] for tagItem in x])
# myDf["tag"] = myDf["tag"].apply(lambda x: [[0 if tagI != tagIndexDict[tagItem] else 1
#                                             for tagI in range(len(tagIndexDict))]
#                                             for tagItem in x])
y=np.array(myDf["tag"].values.tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<?, ?it/s]


In [20]:
myModel=MyBiLSTMCRF(vocabSize,maxLen, tagIndexDict,tagSum,sequenceLengths)

In [21]:
myModel.myBiLSTMCRF.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 100)           1200      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 19, 5)             4240      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 19, 5)             440       
_________________________________________________________________
crf_layer (CRF)              (None, 19)                65        
Total params: 5,945
Trainable params: 5,945
Non-trainable params: 0
_________________________________________________________________


In [22]:
history=myModel.fit(X,y,epochs=1500)

Epoch 1/1500
1/1 [==============================] - 0s 212ms/step - loss: 34.1606
Epoch 2/1500
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1/1 [==============================] - 0s 246ms/step - loss: 34.0737
Epoch 3/1500
1/1 [==============================] - 0s 218ms/step - loss: 33.9872
Epoch 4/1500
1/1 [==============================] - 0s 134ms/step - loss: 33.9009
Epoch 5/1500
1/1 [==============================] - 0s 174ms/step - loss: 33.8148
Epoch 6/1500
1/1 [==============================] - 0s 146ms/step - loss: 33.7288
Epoch 7/1500
1/1 [==============================] - 0s 216ms/step - loss: 33.6428
Epoch 8/1500
1/1 [==============================] - 0s 167ms/step - loss: 33.5566
Epoch 9/1500
1/1 [==============================] - 0s 180ms/step - loss: 33.4703
Epoch 10/1500
1/1 [==============================] - 0s 265ms/step - loss: 33.3837
Epoch 11/1500
1/1 [==============================] - 0s 205ms/step - loss: 33.2966
Epoch 12/1500
1/1 [=====

1/1 [==============================] - 0s 133ms/step - loss: 22.9016
Epoch 97/1500
1/1 [==============================] - 0s 152ms/step - loss: 22.7844
Epoch 98/1500
1/1 [==============================] - 0s 135ms/step - loss: 22.6678
Epoch 99/1500
1/1 [==============================] - 0s 136ms/step - loss: 22.5519
Epoch 100/1500
1/1 [==============================] - 0s 136ms/step - loss: 22.4367
Epoch 101/1500
1/1 [==============================] - 0s 156ms/step - loss: 22.3223
Epoch 102/1500
1/1 [==============================] - 0s 159ms/step - loss: 22.2088
Epoch 103/1500
1/1 [==============================] - 0s 157ms/step - loss: 22.0962
Epoch 104/1500
1/1 [==============================] - 0s 186ms/step - loss: 21.9847
Epoch 105/1500
1/1 [==============================] - 0s 152ms/step - loss: 21.8742
Epoch 106/1500
1/1 [==============================] - 0s 146ms/step - loss: 21.7649
Epoch 107/1500
1/1 [==============================] - 0s 139ms/step - loss: 21.6567
Epoch 108/

1/1 [==============================] - 0s 135ms/step - loss: 15.1155
Epoch 194/1500
1/1 [==============================] - 0s 134ms/step - loss: 15.0619
Epoch 195/1500
1/1 [==============================] - 0s 189ms/step - loss: 15.0086
Epoch 196/1500
1/1 [==============================] - 0s 140ms/step - loss: 14.9557
Epoch 197/1500
1/1 [==============================] - 0s 169ms/step - loss: 14.9031
Epoch 198/1500
1/1 [==============================] - ETA: 0s - loss: 14.85 - 0s 134ms/step - loss: 14.8507
Epoch 199/1500
1/1 [==============================] - 0s 148ms/step - loss: 14.7988
Epoch 200/1500
1/1 [==============================] - 0s 146ms/step - loss: 14.7471
Epoch 201/1500
1/1 [==============================] - 0s 143ms/step - loss: 14.6957
Epoch 202/1500
1/1 [==============================] - 0s 132ms/step - loss: 14.6446
Epoch 203/1500
1/1 [==============================] - 0s 142ms/step - loss: 14.5938
Epoch 204/1500
1/1 [==============================] - 0s 177ms/step

1/1 [==============================] - 0s 138ms/step - loss: 10.9967
Epoch 291/1500
1/1 [==============================] - 0s 172ms/step - loss: 10.9653
Epoch 292/1500
1/1 [==============================] - 0s 187ms/step - loss: 10.9342
Epoch 293/1500
1/1 [==============================] - 0s 155ms/step - loss: 10.9032
Epoch 294/1500
1/1 [==============================] - 0s 137ms/step - loss: 10.8726
Epoch 295/1500
1/1 [==============================] - 0s 109ms/step - loss: 10.8422
Epoch 296/1500
1/1 [==============================] - 0s 137ms/step - loss: 10.8120
Epoch 297/1500
1/1 [==============================] - 0s 206ms/step - loss: 10.7820
Epoch 298/1500
1/1 [==============================] - 0s 187ms/step - loss: 10.7523
Epoch 299/1500
1/1 [==============================] - 0s 225ms/step - loss: 10.7228
Epoch 300/1500
1/1 [==============================] - 0s 162ms/step - loss: 10.6935
Epoch 301/1500
1/1 [==============================] - ETA: 0s - loss: 10.66 - 0s 151ms/step

Epoch 388/1500
1/1 [==============================] - 0s 134ms/step - loss: 8.7618
Epoch 389/1500
1/1 [==============================] - 0s 177ms/step - loss: 8.7451
Epoch 390/1500
1/1 [==============================] - 0s 193ms/step - loss: 8.7285
Epoch 391/1500
1/1 [==============================] - 0s 155ms/step - loss: 8.7119
Epoch 392/1500
1/1 [==============================] - 0s 130ms/step - loss: 8.6955
Epoch 393/1500
1/1 [==============================] - 0s 124ms/step - loss: 8.6791
Epoch 394/1500
1/1 [==============================] - 0s 108ms/step - loss: 8.6628
Epoch 395/1500
1/1 [==============================] - 0s 152ms/step - loss: 8.6465
Epoch 396/1500
1/1 [==============================] - 0s 132ms/step - loss: 8.6304
Epoch 397/1500
1/1 [==============================] - 0s 144ms/step - loss: 8.6143
Epoch 398/1500
1/1 [==============================] - 0s 228ms/step - loss: 8.5983
Epoch 399/1500
1/1 [==============================] - 0s 139ms/step - loss: 8.5824
Epoc

...

Epoch 1074/1500
1/1 [==============================] - 0s 212ms/step - loss: 3.9664
Epoch 1075/1500
1/1 [==============================] - 0s 177ms/step - loss: 3.9633
Epoch 1076/1500
1/1 [==============================] - 0s 182ms/step - loss: 3.9602
Epoch 1077/1500
1/1 [==============================] - 0s 183ms/step - loss: 3.9571
Epoch 1078/1500
1/1 [==============================] - 0s 231ms/step - loss: 3.9540
Epoch 1079/1500
1/1 [==============================] - 0s 220ms/step - loss: 3.9510
Epoch 1080/1500
1/1 [==============================] - 0s 186ms/step - loss: 3.9479
Epoch 1081/1500
1/1 [==============================] - 0s 281ms/step - loss: 3.9449
Epoch 1082/1500
1/1 [==============================] - 0s 138ms/step - loss: 3.9418
Epoch 1083/1500
1/1 [==============================] - 0s 164ms/step - loss: 3.9388
Epoch 1084/1500
1/1 [==============================] - 0s 143ms/step - loss: 3.9357
Epoch 1085/1500
1/1 [==============================] - 0s 128ms/step - loss:

1/1 [==============================] - 0s 326ms/step - loss: 3.6892
Epoch 1172/1500
1/1 [==============================] - 1s 1s/step - loss: 3.6866
Epoch 1173/1500
1/1 [==============================] - 1s 683ms/step - loss: 3.6839
Epoch 1174/1500
1/1 [==============================] - 0s 387ms/step - loss: 3.6813
Epoch 1175/1500
1/1 [==============================] - 0s 299ms/step - loss: 3.6787
Epoch 1176/1500
1/1 [==============================] - 0s 280ms/step - loss: 3.6760
Epoch 1177/1500
1/1 [==============================] - 0s 307ms/step - loss: 3.6734
Epoch 1178/1500
1/1 [==============================] - 0s 310ms/step - loss: 3.6708
Epoch 1179/1500
1/1 [==============================] - 0s 271ms/step - loss: 3.6682
Epoch 1180/1500
1/1 [==============================] - 0s 267ms/step - loss: 3.6656
Epoch 1181/1500
1/1 [==============================] - 0s 264ms/step - loss: 3.6630
Epoch 1182/1500
1/1 [==============================] - 0s 359ms/step - loss: 3.6604
Epoch 1183/

1/1 [==============================] - 1s 1s/step - loss: 3.4459
Epoch 1269/1500
1/1 [==============================] - 0s 471ms/step - loss: 3.4434
Epoch 1270/1500
1/1 [==============================] - 0s 431ms/step - loss: 3.4409
Epoch 1271/1500
1/1 [==============================] - 0s 325ms/step - loss: 3.4384
Epoch 1272/1500
1/1 [==============================] - 0s 348ms/step - loss: 3.4359
Epoch 1273/1500
1/1 [==============================] - 0s 268ms/step - loss: 3.4334
Epoch 1274/1500
1/1 [==============================] - 0s 378ms/step - loss: 3.4309
Epoch 1275/1500
1/1 [==============================] - 0s 228ms/step - loss: 3.4284
Epoch 1276/1500
1/1 [==============================] - 0s 375ms/step - loss: 3.4258
Epoch 1277/1500
1/1 [==============================] - 0s 329ms/step - loss: 3.4233
Epoch 1278/1500
1/1 [==============================] - 0s 327ms/step - loss: 3.4207
Epoch 1279/1500
1/1 [==============================] - 0s 341ms/step - loss: 3.4181
Epoch 1280/

1/1 [==============================] - 0s 268ms/step - loss: 3.1641
Epoch 1366/1500
1/1 [==============================] - 0s 130ms/step - loss: 3.1614
Epoch 1367/1500
1/1 [==============================] - 0s 199ms/step - loss: 3.1588
Epoch 1368/1500
1/1 [==============================] - 0s 237ms/step - loss: 3.1561
Epoch 1369/1500
1/1 [==============================] - 0s 218ms/step - loss: 3.1534
Epoch 1370/1500
1/1 [==============================] - 0s 301ms/step - loss: 3.1508
Epoch 1371/1500
1/1 [==============================] - 0s 169ms/step - loss: 3.1481
Epoch 1372/1500
1/1 [==============================] - 0s 152ms/step - loss: 3.1455
Epoch 1373/1500
1/1 [==============================] - 0s 165ms/step - loss: 3.1428
Epoch 1374/1500
1/1 [==============================] - 0s 142ms/step - loss: 3.1402
Epoch 1375/1500
1/1 [==============================] - 0s 212ms/step - loss: 3.1376
Epoch 1376/1500
1/1 [==============================] - 0s 136ms/step - loss: 3.1349
Epoch 13

1/1 [==============================] - 0s 123ms/step - loss: 2.9160
Epoch 1463/1500
1/1 [==============================] - 0s 139ms/step - loss: 2.9136
Epoch 1464/1500
1/1 [==============================] - 0s 132ms/step - loss: 2.9111
Epoch 1465/1500
1/1 [==============================] - 0s 170ms/step - loss: 2.9086
Epoch 1466/1500
1/1 [==============================] - 0s 134ms/step - loss: 2.9061
Epoch 1467/1500
1/1 [==============================] - 0s 127ms/step - loss: 2.9037
Epoch 1468/1500
1/1 [==============================] - 0s 133ms/step - loss: 2.9012
Epoch 1469/1500
1/1 [==============================] - 0s 123ms/step - loss: 2.8987
Epoch 1470/1500
1/1 [==============================] - 0s 137ms/step - loss: 2.8963
Epoch 1471/1500
1/1 [==============================] - 0s 137ms/step - loss: 2.8938
Epoch 1472/1500
1/1 [==============================] - 0s 167ms/step - loss: 2.8913
Epoch 1473/1500
1/1 [==============================] - 0s 175ms/step - loss: 2.8889
Epoch 14

In [23]:
testI=2
preY=myModel.predict(X)[testI]

In [24]:
indexTagDict=dict(list(zip(list(tagIndexDict.values()),list(tagIndexDict.keys()))))
indexWordDict=dict(list(zip(list(wordIndexDict.values()),list(wordIndexDict.keys()))))

sentenceList=[indexWordDict[wordItem] for wordItem in X[testI]]
sentenceList=sentenceList[:sentenceList.index("<end>")]

tagList=[indexTagDict[tagItem] for tagItem in preY]
tagList=tagList[:tagList.index("END")]

print(" ".join(sentenceList))
print(" ".join(tagList))

将 将 军 带 上 战 车
S B I B I B I


In [25]:
myDf.head()

,text,tag
0,"[1, 2, 3, 4, 5, 6]","[1, 2, 1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 2, 7, 8, 9, 4, 10, 6]","[3, 1, 2, 1, 2, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 1, 2, 8, 9, 4, 11, 6]","[3, 1, 2, 1, 2, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[2, 4, 1, 2, 6]","[3, 3, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[1, 2, 4, 1, 2, 6]","[3, 3, 3, 1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
